# Robot Localization
[Prob. aplicada a robots](https://www.notion.so/Probabilidades-aplicadas-a-la-rob-tica-1d6c5fad35f04cdcaa6781121da341e7)



In [2]:
import numpy as np

In [13]:
# Programar para un número variable de celdas una distribución uniforme
p=[]
n=5
p = np.ones(n) / n
#...
print(p)

[0.2 0.2 0.2 0.2 0.2]


## Probabildad después de la medición

Consideremos un mundo con cinco celdas, $x_1$ a $x_5$. Asumamos que 2 de las celdas son de color rojo y el resto de color verde.

>> mundo = ['verde','rojo','rojo','verde','verde']

cada celda sigue teniendo probabilidad uniforme

>> p = [0.2, 0.2, 0.2, 0.2, 0.2]

Supongamos que el robot detecta el rojo. Vamos a ver una forma simple de incorporar la medición en nuestra *creencia*, usando el producto. Cada vez que la celda coincida con la medición (celda de color rojo) se va a multiplicar por .6, mientras que si la celda es verde, se multiplica por .2.

Como se modificarían las probabilidades en este caso? (editar este texto y completar)

>> p=[0.04, 0.12, 0.12, 0.04, 0.04]

Van a ver que las probabilidades no suman 1. Por eso vamos a ver una forma de renormalizar.


## Normalizando la distribución

Como habrán observado, la suma de las probabilidades de cada celda después de la medición no suma 1. Para que esto ahora sea una distribución es necesario dividir cada celda por la suma de las celdas.

Como sería la nueva distribución de probabilidad una vez que se dividió por el total anterior?

>> p_N = [0.11,0.33,0.33,0.11,0.11]

Revisar que la suma de cada celda da 1.

Con qué probabilidades el robot *cree* que está en cada celda? Porque sucede que el robot no tiene seguridad absoluta de estar en una celda?

## Función Sense

In [22]:
p = [0.2,0.2,0.2,0.2,0.2]
# Vamos a introducir una variable llamada mundo
mundo = ['green','red','red','green','green']
# Y otra variable de medición Z
Z = 'red'
pHit = 0.6   # p(z_{red} | pi) 
pMiss = 0.2  # p( z_{greend} | pi)
# Defina una función sense que me devuelva la probabilidad normalizada q en función de las mediciones

def sense(p,Z):
  q = [pHit if (x == Z) else pMiss for x in mundo]
  q = np.array(q) * p
  q /= np.sum(q)
  return q

print(sense(p,Z))


[0.11111111 0.33333333 0.33333333 0.11111111 0.11111111]


In [23]:
# Ahora supongamos que en vez de una medición tenemos varias mediciones (en principio 2)
measurements = ['red','green']
# Sin modificar la función sense.
# Qué agregarían para modificar la probabilidad a posteriori para cada una de las mediciones?
# Tu código
p_aux = p
for measure in measurements:
  p_aux = sense(p_aux, measure)
  print(p_aux)
# Para el caso de las mediciones que les dimos. Porqué se obtiene este resultado?

[0.11111111 0.33333333 0.33333333 0.11111111 0.11111111]
[0.2 0.2 0.2 0.2 0.2]


**En lo anterior medimos y tuvimos una puerta roja; despues SIN MOVERME, medí verde, perdi toda la info que tenia y vuelvo a la uniforme**

# Movimiento

## Movimiento Exacto
Supongamos que tenemos una distribución en las 5 celdas de: [1/9][1/3][1/3][1/9][1/9]. Sin saber donde se encuentra el robot, este se mueve hacia la derecha. Se considerará que el mundo es cíclico, es decir, que cuando llegue a la última celda va a regresar a la primera. Cómo serán las probabilidades si el robot se mueve un lugar hacia la derecha?

[1/9][1/9][1/3][1/3][1/9]

**LAS CELDAS SE MUEVEN CON EL ROBOT**



In [35]:
from collections import deque
# Definamos una función de movimiento exacto que represente lo anterior. 
p =  [0,1,0,0,0]
U = 1

def move(p, U):
  ''' p: distribución
      U: numero de celdas que se mueve (Cuando es positivo mueve hacia la derecha)'''
  a_list = deque(p)
  a_list.rotate(U)
  q = list(a_list)
  return q

print(move(p,U))
      

[0, 0, 1, 0, 0]


## Movimiento Inexacto (Real)
En la realidad, el movimiento del robot está sujeto a ruido. Las velocidades de movimiento difieren de las envíadas por el comando. Supongamos que queremos movernos en U=2, pero el robot tiene las siguientes probabilidades a priori para su movimiento:

$P(x_{i+U}|X_i)=0.8$

$P(x_{i+(U-1)}|X_i)=0.1$ 

$P(x_{i+(U+1)}|X_i)=0.1$ 

Si el robot comienza desde la siguiente posición: [0][1][0][0][0] y se mueve en U=2. Cómo queda la distribución?

[0][0][0.1][0.8][0.1]

Fijense que el movimiento agregó incertidumrbe a la posición del robot.

### Otros ejemplos
Que sucede ahora si la distribución a priori es:

[0][0.5][0][0.5][0]

El robot se mueve en U=2 con las mismas probabilidades a priori que antes:

- [0.4] [0.05] [0] [0] [0.05]
- [0] [0] [0.05] [0.4] [0.05] 


- [0.4] [0.05] [0.05] [0.4] [0.1]

Recordemos que estamos considerando un mundo cíclico.


---

Y si la distribución fuera uniforme?

[0.2][0.2][0.2][0.2][0.2]

Como sería la distribución luego de moverse en U=2?

[0.2][0.2][0.2][0.2][0.2]

**HACER EL CALCULO PARA VER PORQUE PASA ESTO**



## Función move inexacta

In [54]:
# Modificar la función move con tal de agregar las probabilidades
# de overshooting y undershooting al destino deseado.

# Traten de mantener la función move para que haga solamente un movimiento
# cada vez que es llamada.
p = [0,1,0,0,0]
pExact = 0.8
pOvershoot = 0.1
pUndershoot = 0.1

def move(p,U):
  q= np.zeros(len(p))

  i = U - 1
  a_list = deque(p)
  a_list.rotate(i)
  q += np.array(a_list) * pUndershoot

  i = U
  a_list = deque(p)
  a_list.rotate(i)
  q += np.array(a_list) * pExact

  i = U + 1
  a_list = deque(p)
  a_list.rotate(i)
  q += np.array(a_list) * pOvershoot

  return q

print(move(p,2))

# Que sucede si el robot continúa moviendose en U=1 sin sensar donde está parado?
for i in range(100):
  p = move(p,1)

print(p)

[0.  0.  0.1 0.8 0.1]
[0.20000004 0.20000014 0.20000004 0.19999989 0.19999989]


**Al hacer muchos movimientos sin sensar, y hacer movimientos inexactos, pierdo información. Llega un momento donde no se más donde estoy (distribución uniforme)**

# Sense and Move


[Pizarra](https://jamboard.google.com/d/1_hrIzxYzVzBlikjVzoWb4Bn4ortIB128wx13MfWvlqc/viewer)



In [58]:
# Con la lista de movimientos dados 
# motions = [1,1] que significa que el robot se mueve dos veces hacia la derecha
# calcular la probabilidad a posteriori si el robot
# primero mide rojo, luego se mueve, luego mide verde y se mueve de nuevo.
# Comenzando con una distribución a priori uniforme.

p=[0.2, 0.2, 0.2, 0.2, 0.2]
world=['green', 'red', 'red', 'green', 'green']
measurements = ['red', 'green']
motions = [1,1]
pHit = 0.6
pMiss = 0.2
pExact = 0.8
pOvershoot = 0.1
pUndershoot = 0.1

def sense(p,Z):
  q = [pHit if (x == Z) else pMiss for x in mundo]
  q = np.array(q) * p
  q /= np.sum(q)
  return q

def move(p,U):
  q= np.zeros(len(p))

  i = U - 1
  a_list = deque(p)
  a_list.rotate(i)
  q += np.array(a_list) * pUndershoot

  i = U
  a_list = deque(p)
  a_list.rotate(i)
  q += np.array(a_list) * pExact

  i = U + 1
  a_list = deque(p)
  a_list.rotate(i)
  q += np.array(a_list) * pOvershoot

  return q

#
# ADD CODE HERE
#
for i in range(len(motions)):
  p  = sense(p, measurements[i])
  print(p)
  p = move(p , motions[i])
  print(p)

print("----------------------")

# Analizar los resultados observando las mediciones y el mundo.
# Ahora hacer el mismo ejemplo con
# measurements = ['red','red']
p=[0.2, 0.2, 0.2, 0.2, 0.2]
measurements = ['red', 'red']
motions = [1,1]
for i in range(len(motions)):
  p  = sense(p, measurements[i])
  print(p)
  p = move(p , motions[i])
  print(p)

[0.11111111 0.33333333 0.33333333 0.11111111 0.11111111]
[0.11111111 0.13333333 0.31111111 0.31111111 0.13333333]
[0.15789474 0.06315789 0.14736842 0.44210526 0.18947368]
[0.21157895 0.15157895 0.08105263 0.16842105 0.38736842]
----------------------
[0.11111111 0.33333333 0.33333333 0.11111111 0.11111111]
[0.11111111 0.13333333 0.31111111 0.31111111 0.13333333]
[0.05882353 0.21176471 0.49411765 0.16470588 0.07058824]
[0.07882353 0.07529412 0.22470588 0.43294118 0.18823529]
